In [1]:
!pip install fiftyone
!pip install transformers
!pip install super-gradients

You should consider upgrading via the 'C:\Users\anssi\Desktop\ai\env\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\anssi\Desktop\ai\env\Scripts\python.exe -m pip install --upgrade pip' command.


  Using cached super_gradients-3.5.0-py3-none-any.whl (12.0 MB)
  Using cached pyparsing-2.4.5-py2.py3-none-any.whl (67 kB)
  Using cached Sphinx-4.0.3-py3-none-any.whl (2.9 MB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached sphinx_rtd_theme-2.0.0-py2.py3-none-any.whl (2.8 MB)
  Using cached protobuf-3.20.3-cp310-cp310-win_amd64.whl (904 kB)
  Using cached pip_tools-7.3.0-py3-none-any.whl (57 kB)
  Using cached treelib-1.6.1-py3-none-any.whl
  Using cached jsonschema-4.20.0-py3-none-any.whl (84 kB)
  Using cached coverage-5.3.1-py3-none-any.whl
  Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl (38.1 MB)
  Using cached data_gradients-0.3.1-py3-none-any.whl (459 kB)
  Using cached stringcase-1.2.0-py3-none-any.whl
  Using cached rapidfuzz-3.5.2-cp310-cp310-win_amd64.whl (1.7 MB)
  Using cached json_tricks-3.16.1-py2.py3-none-any.whl (27 kB)
  Using cached torchmetrics-0.8.0-py3-none-any.whl (408 kB)
  Using cached pycocotools-2.0.6.tar.gz (24 kB)
  Installing 

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [17 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\pycocotools
  copying pycocotools\coco.py -> build\lib.win-amd64-cpython-310\pycocotools
  copying pycocotools\cocoeval.py -> build\lib.win-amd64-cpython-310\pycocotools
  copying pycocotools\mask.py -> build\lib.win-amd64-cpython-310\pycocotools
  copying pycocotools\__init__.py -> build\lib.win-amd64-cpython-310\pycocotools
  running build_ext
  C:\Users\anssi\AppData\Local\Temp\pip-build-env-va9wcf4x\overlay\Lib\site-packages\Cython\Compiler\Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: C:\Users\anssi\AppData\Local\Temp\pip-install-vvnbxjrj\pycocotools_2681990a7d4d4535b020f4ab84531ad0\pycocotools\_mask.pyx
    tree = Parsing.p_module(s, pxd, full_m

In [2]:
!pip install torch torchvision

You should consider upgrading via the 'C:\Users\anssi\Desktop\ai\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import torch
import torchvision
from transformers import DetrForObjectDetection, DetrImageProcessor
# Run the model on GPU if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load a pre-trained Faster R-CNN model
#model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model.to(device)
model.eval()

print("Model ready")

C:\Users\anssi\Desktop\ai\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Model ready


In [9]:
import fiftyone as fo
import fiftyone.zoo as foz

#dataset = foz.load_zoo_dataset(
 #   "coco-2017",
  #  split="validation",
   # dataset_name="evaluate-detections-tutorial",
#)
name="coco-patched-12345"
dataset_dir="./coco_adv_patch/validation/"
dataset_type = fo.types.COCODetectionDataset
dataset = fo.Dataset.from_dir(dataset_dir, dataset_type, name=name)
dataset.persistent = False
predictions_view = dataset.take(20, seed=51)

   0% |/--------------|    1/5000 [1.8ms elapsed, 8.9s remaining, 39.4K samples/s]     


KeyError: '000000000139.jpg'

In [ ]:
# Print some information about the dataset
print(dataset)

In [ ]:
# Print a ground truth detection
sample = dataset.first()
#print(sample.detections)

In [ ]:
session = fo.launch_app(dataset)

In [ ]:
# Choose a random subset of 100 samples to add predictions to
#predictions_view = dataset.take(100, seed=51)

In [ ]:
from PIL import Image
from torchvision.transforms import functional as func

import fiftyone as fo

# Get class list
classes = dataset.default_classes

# Add predictions to samples
for sample in predictions_view:
    image = Image.open(sample.filepath)

    # you can specify the revision tag if you don't want the timm dependency
    image_as_tensor = func.to_tensor(image).to(device)

    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    c, h, w = image_as_tensor.shape

    # convert outputs (bounding boxes and class logits) to COCO API
    # let's only keep detections with score > 0.9
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]
    detections = []
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = [round(i, 2) for i in box.tolist()]
        #print(
         #       f"Detected {model.config.id2label[label.item()]} with confidence "
          #      f"{round(score.item(), 3)} at location {box}"
        #)
        x1, y1, x2, y2 = box
        rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]
        
    #for label, score, box in zip(labels, scores, boxes):
        # Convert to [top-left-x, top-left-y, width, height]
        # in relative coordinates in [0, 1] x [0, 1]
     #   x1, y1, x2, y2 = box
      #  rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]

        detections.append(
            fo.Detection(
                label=model.config.id2label[label.item()],
                bounding_box=rel_box,
                confidence=score.item()
            )
        )

    # Save predictions to dataset
    sample["detr"] = fo.Detections(detections=detections)
    sample.save()
print(detections)
print(box)
print(score.item())
print("Finished adding predictions")

In [ ]:
session.view = predictions_view

In [ ]:
from fiftyone import ViewField as F

# Only contains detections with confidence >= 0.75
high_conf_view = predictions_view.filter_labels("detr", F("confidence") > 0.75, only_matches=False)

In [ ]:
# Print some information about the view
print(high_conf_view)

In [ ]:
# Print a prediction from the view to verify that its confidence is > 0.75
sample = high_conf_view.first()
print(sample.detr.detections[0])

In [ ]:
# Load high confidence view in the App
session.view = high_conf_view

In [ ]:
session.view = predictions_view

In [ ]:
session.view = predictions_view

In [ ]:
# Evaluate the predictions in the `detr` field of our `high_conf_view`
# with respect to the objects in the `ground_truth` field
results = high_conf_view.evaluate_detections(
    "detr",
    gt_field="detections",
    eval_key="detr_eval",
    compute_mAP=True,
)
print(results)

In [ ]:
# Get the 10 most common classes in the dataset
counts = dataset.count_values("detections.detections.label")
classes_top10 = sorted(counts, key=counts.get, reverse=True)[:10]

# Print a classification report for the top-10 classes
results.print_report(classes=classes_top10)

In [ ]:
print(results.mAP())